In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    #"red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    #"red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
    #          {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,"num_envs_per_worker":1,
          "train_batch_size": 600*5,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-10 00:36:19,625	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-10 00:36:19,627	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-10 00:36:38,717	INFO trainable.py:130 -- Trainable.setup took 19.094 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-10 00:36:38,719	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-10 00:36:55,947	INFO trainable.py:130 -- Trainable.setup took 17.171 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-10 00:36:55,948	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)

                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    save_weights("blue_0",trainer)
                    save_weights("blue_1",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

1
-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
2
-------------------------- Scene: 0 --------------------------
1
-------------------------- Scene: 0 --------------------------
88 blue_0 DOWN
497 red_0 Shoot at blue_1
508 red_0 Shoot at blue_1
605 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 605 -0.10099964666666668 -1.4054923600000002
blue_1 False False 605 -0.14083694666666666 -6.932175120000004
2
-------------------------- Scene: 0 --------------------------
158 blue_1 Shoot at red_1 launch distance : 57479.34787382334 True True
169 blue_1 Shoot at red_1 launch distance : 51949.94177090096 True True
Same tgt shoot
Same tgt shoot
204 blue_0 Shoot at red_0 launch distance : 58419.78796606506 True True
211 red_0 Shoot at blue_1
Same tgt shoot
Same tgt shoot
216 blue_0 Shoot at red_0 launch distance : 53588.781717818514 True True
222 red_0 Shoot at blue_1
262 blue_1 Splash :red_1
277 blue_1: Destro

2022-04-10 00:44:03,778	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


agent_timesteps_total: 6638
custom_metrics: {}
date: 2022-04-10_00-44-03
done: false
episode_len_mean: 829.75
episode_media: {}
episode_reward_max: 236.2535410316475
episode_reward_mean: -2.4279524687548317
episode_reward_min: -149.8922920966669
episodes_this_iter: 4
episodes_total: 4
experiment_id: 825995779ae848a3adbe36f16056c84f
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.816363334655762
        entropy_coeff: 0.0
        kl: 0.13412289321422577
        model: {}
        policy_loss: -0.02197049930691719
        total_loss: 339.0068664550781
        vf_explained_var: 0.01265669148415327
        vf_loss: 339.00201416015625
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.480523586273193
        entropy_coeff: 0.0
        kl: 0.28919708728790283
        model: {}
 

2022-04-10 00:44:04,504	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000001\checkpoint-1
2022-04-10 00:44:04,505	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': 3319, '_time_total': 419.713485956192, '_episodes_total': 4}


3
-------------------------- Scene: 0 --------------------------
199 blue_1 Shoot at red_0 launch distance : 59879.05211340607 True True
210 blue_1 Shoot at red_0 launch distance : 54216.16120678409 True True
259 red_1 Shoot at blue_1
270 red_1 Shoot at blue_1
314 blue_1 Splash :red_0
338 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.89299996 -70.83712740333345
blue_1 False False 1200 0.89899996 115.64428383857746

----------------- Training at steps:1 start! -----------------
4
-------------------------- Scene: 0 --------------------------
288 blue_1 Shoot at red_0 launch distance : 56600.62344179612 True True
299 blue_1 Shoot at red_0 launch distance : 51248.88804647375 True True
337 red_1 Shoot at blue_1
348 red_1 Shoot at blue_1
398 blue_1 Splash :red_0
411 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.89299996 -87.01811293666695
blue_1 False False 1200 0.89899996 120.37549986376887
6
-------------------------- Scene: 0 --------------------------
144 blu

...

agent_timesteps_total: 476890
custom_metrics: {}
date: 2022-04-10_11-02-11
done: false
episode_len_mean: 393.11
episode_media: {}
episode_reward_max: 269.1352376873101
episode_reward_mean: 42.02241435629964
episode_reward_min: -60.13305699666665
episodes_this_iter: 7
episodes_total: 488
experiment_id: 825995779ae848a3adbe36f16056c84f
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 19.648286819458008
        entropy_coeff: 0.0
        kl: 1.4993443489074707
        model: {}
        policy_loss: 0.0660938248038292
        total_loss: 25.68043327331543
        vf_explained_var: -0.23491300642490387
        vf_loss: 25.314472198486328
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 19.05815887451172
        entropy_coeff: 0.0
        kl: 0.846798300743103
        model: {}
  

2022-04-10 11:11:10,305	INFO trainable.py:496 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000072\checkpoint-72


agent_timesteps_total: 483410
custom_metrics: {}
date: 2022-04-10_11-11-10
done: false
episode_len_mean: 399.21
episode_media: {}
episode_reward_max: 269.1352376873101
episode_reward_mean: 42.820915851561466
episode_reward_min: -60.13305699666665
episodes_this_iter: 6
episodes_total: 494
experiment_id: 825995779ae848a3adbe36f16056c84f
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 17.161741256713867
        entropy_coeff: 0.0
        kl: 0.8323363661766052
        model: {}
        policy_loss: 0.24272902309894562
        total_loss: 238.07240295410156
        vf_explained_var: -0.41081860661506653
        vf_loss: 237.66317749023438
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 21.6643009185791
        entropy_coeff: 0.0
        kl: 1.755181908607483
        model: {}


2022-04-10 11:11:10,306	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 72, '_timesteps_total': 241705, '_time_total': 31835.76479458809, '_episodes_total': 494}


145
-------------------------- Scene: 0 --------------------------
331 blue_0 Shoot at red_0 launch distance : 59945.0509133156 True True
342 blue_0 Shoot at red_0 launch distance : 55519.91070778122 True True
